In [1]:
from sklearn.datasets import load_iris

In [2]:
iris=load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)

In [5]:
df.head(4)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
X=df

In [8]:
X.head()
X += np.random.normal(0, 0.5, X.shape) #some noise are added to understand gridsearchCV and randomsearchCV effectively

In [9]:
y=iris.target

In [10]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
rfc=RandomForestClassifier()

In [13]:
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

In [14]:
accuracy_score(y_test,y_pred)

0.8333333333333334

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred_lr=lr.predict(X_test)
accuracy_score(y_test,y_pred_lr)

0.8333333333333334

In [17]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [28]:
param_grid = {
    'n_estimators': [50, 100],        # was 4 values → now 2
    'max_depth': [None, 10],          # was 4 → now 2
    'min_samples_split': [2, 5],      # was 3 → now 2
    'min_samples_leaf': [1, 2],       # was 3 → now 2
    'max_features': ['sqrt']          # was 2 → now 1
}


In [30]:
grid_rfc = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=2,
    scoring='accuracy',
    n_jobs=-1
)


In [31]:
grid_rfc.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100]},
             scoring='accuracy')

In [32]:

print("Best Parameters:", grid_rfc.best_params_)
print("Best CV Score:", grid_rfc.best_score_)

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best CV Score: 0.8666666666666667


In [33]:
best_rfc = grid_rfc.best_estimator_
y_pred_best = best_rfc.predict(X_test)
print("Tuned Test Accuracy:", accuracy_score(y_test, y_pred_best))


Tuned Test Accuracy: 0.8333333333333334


In [34]:
from scipy.stats import randint


In [35]:
param_dist = {
    'n_estimators': randint(50, 400),
    'max_depth': [None] + list(range(5, 25)),
    'min_samples_split': randint(2, 15),
    'min_samples_leaf': randint(1, 8),
    'max_features': ['sqrt', 'log2', None]
}


In [36]:
random_rfc = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

In [37]:
random_rfc.fit(X_train, y_train)
print("Best Random Params:", random_rfc.best_params_)
print("Best Random CV Score:", random_rfc.best_score_)

Best Random Params: {'max_depth': 24, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 264}
Best Random CV Score: 0.9


In [38]:
results = pd.DataFrame({
    "Model": ["Default RF", "GridSearch RF", "RandomSearch RF"],
    "Accuracy": [
        accuracy_score(y_test, rfc.predict(X_test)),
        accuracy_score(y_test, best_rfc.predict(X_test)),
        accuracy_score(y_test, random_rfc.best_estimator_.predict(X_test))
    ]
})

print(results)


             Model  Accuracy
0       Default RF  0.833333
1    GridSearch RF  0.833333
2  RandomSearch RF  0.800000
